# Лабораторная работа №1
## Вариант 40. Кибербезопасность - Анализ логов доступа

**Предметная область.** Анализ безопасности веб-приложений через мониторинг логов доступа

**Бизнес-задачи:**
1. Загрузить и обработать логи доступа (Access Logs) в HDFS
2. Фильтровать внешние IP-адреса для анализа
3. Выявить аномальные паттерны доступа (подозрение на DDoS/брутфорс)
4. Визуализировать временные ряды с выделением аномалий

---

## 1. Подготовка окружения

In [ ]:
# Установка необходимых библиотек
!pip install pyspark matplotlib seaborn pandas numpy

In [ ]:
# Импорт библиотек
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, avg, stddev, hour, to_timestamp,
    regexp_extract, when, lit, sum as spark_sum,
    date_format, window, desc, asc
)
from pyspark.sql.types import (
    StructType, StructField, StringType, 
    IntegerType, TimestampType
)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 2. Инициализация Spark Session

In [ ]:
# Создание SparkSession с подключением к HDFS
spark = SparkSession.builder \
    .appName("Variant40_Cybersecurity_AccessLogs") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://localhost:9000") \
    .config("spark.ui.port", "4040") \
    .config("spark.sql.shuffle.partitions", "50") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Установка уровня логирования
spark.sparkContext.setLogLevel("WARN")

print(f"Spark Version: {spark.version}")
print(f"Spark UI: http://localhost:4040")

## 3. Генерация синтетических данных (Access Logs)

Создадим реалистичные логи доступа веб-сервера с нормальным трафиком и аномалиями

In [ ]:
import random
import csv
from datetime import datetime, timedelta

def generate_access_logs(filename='/tmp/access_logs.csv', num_records=50000):
    """
    Генерация синтетических логов доступа с аномалиями
    """
    
    # IP-адреса (внутренние и внешние)
    internal_ips = [f"192.168.1.{i}" for i in range(1, 50)]
    normal_external_ips = [f"{random.randint(1, 223)}.{random.randint(0, 255)}.{random.randint(0, 255)}.{random.randint(1, 254)}" 
                          for _ in range(200)]
    
    # Аномальные IP (DDoS-атака)
    anomalous_ips = [
        "185.220.101.45",  # Известный ботнет
        "198.50.200.135",  # Подозрительный IP
        "103.89.91.122",   # Сканер
        "45.142.212.61"    # Брутфорс
    ]
    
    # URL endpoints
    endpoints = [
        "/api/v1/users",
        "/api/v1/products",
        "/login",
        "/admin",
        "/search",
        "/checkout",
        "/profile",
        "/api/v1/orders"
    ]
    
    # HTTP методы и статус-коды
    methods = ['GET', 'POST', 'PUT', 'DELETE']
    status_codes = [200, 201, 204, 301, 302, 400, 401, 403, 404, 500]
    
    # User Agents
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36",
        "python-requests/2.28.1",
        "curl/7.68.0",
        "Googlebot/2.1",
        "Bot/Scanner/1.0"  # Подозрительный
    ]
    
    records = []
    start_date = datetime(2024, 2, 1, 0, 0, 0)
    
    for i in range(num_records):
        # Временная метка (равномерное распределение за 30 дней)
        timestamp = start_date + timedelta(
            days=random.randint(0, 29),
            hours=random.randint(0, 23),
            minutes=random.randint(0, 59),
            seconds=random.randint(0, 59)
        )
        
        # Определяем тип запроса (90% нормальные, 10% аномальные)
        is_anomaly = random.random() < 0.1
        is_internal = random.random() < 0.15
        
        if is_internal:
            ip = random.choice(internal_ips)
            requests_count = random.randint(1, 20)  # Нормальная активность
        elif is_anomaly:
            ip = random.choice(anomalous_ips)
            # Аномальные IP делают МНОГО запросов
            requests_count = random.randint(500, 2000)
        else:
            ip = random.choice(normal_external_ips)
            requests_count = random.randint(1, 50)
        
        method = random.choice(methods)
        endpoint = random.choice(endpoints)
        
        # Аномальные запросы чаще получают ошибки
        if is_anomaly:
            status_code = random.choice([401, 403, 404, 429, 500])
        else:
            status_code = random.choice([200, 200, 200, 201, 301, 400, 404])
        
        response_time = random.randint(50, 5000) if not is_anomaly else random.randint(1000, 10000)
        user_agent = random.choice(user_agents[:-1]) if not is_anomaly else user_agents[-1]
        
        record = {
            'timestamp': timestamp.strftime('%Y-%m-%d %H:%M:%S'),
            'ip_address': ip,
            'method': method,
            'endpoint': endpoint,
            'status_code': status_code,
            'response_time_ms': response_time,
            'user_agent': user_agent,
            'bytes_sent': random.randint(100, 50000)
        }
        records.append(record)
    
    # Сохранение в CSV
    with open(filename, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=records[0].keys())
        writer.writeheader()
        writer.writerows(records)
    
    print(f"✅ Сгенерировано {len(records)} записей логов")
    print(f"📁 Файл сохранен: {filename}")
    return filename

# Генерация данных
data_file = generate_access_logs()

# Просмотр первых строк
print("\n📊 Первые 5 записей:")
!head -6 /tmp/access_logs.csv

## 4. Задание 1. Загрузка данных в HDFS и фильтрация внешних IP

bash# В терминале от пользователя hadoop:

# 4. Проверьте что файл доступен
ls -lh /tmp/access_logs.csv


# 5. Создайте директорию в HDFS
hdfs dfs -mkdir -p /user/hadoop/lab_01_variant40/input

# 6. Загрузите файл в HDFS
hdfs dfs -put -f /tmp/access_logs.csv /user/hadoop/lab_01_variant40/input/


In [ ]:
# Загрузка данных из HDFS в Spark DataFrame
hdfs_path = "hdfs://localhost:9000/user/hadoop/lab_01_variant40/input/access_logs.csv"

df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(hdfs_path)

print("📊 Схема данных:")
df.printSchema()

print("\n📈 Общая статистика:")
print(f"Всего записей: {df.count():,}")

print("\n🔍 Примеры данных:")
df.show(5, truncate=False)

In [ ]:
# Очистка данных: удаление пустых значений
df_clean = df.dropna()

# Преобразование timestamp в правильный формат
df_clean = df_clean.withColumn(
    'timestamp',
    to_timestamp(col('timestamp'), 'yyyy-MM-dd HH:mm:ss')
)

print(f"✅ Очищенных записей: {df_clean.count():,}")
print(f"🗑️ Удалено записей: {df.count() - df_clean.count():,}")

In [ ]:
# Фильтрация внешних IP (исключаем внутренние сети)
# Внутренние сети: 192.168.x.x, 10.x.x.x, 172.16-31.x.x, 127.x.x.x

df_external = df_clean.filter(
    ~(
        col('ip_address').startswith('192.168.') |
        col('ip_address').startswith('10.') |
        col('ip_address').startswith('127.') |
        col('ip_address').rlike('^172\\.(1[6-9]|2[0-9]|3[0-1])\\.')
    )
)

print("🌐 Статистика по IP-адресам:")
print(f"Всего записей: {df_clean.count():,}")
print(f"Внешние IP: {df_external.count():,}")
print(f"Внутренние IP: {df_clean.count() - df_external.count():,}")

# Агрегация по IP адресам
ip_stats = df_external.groupBy('ip_address') \
    .agg(
        count('*').alias('request_count'),
        avg('response_time_ms').alias('avg_response_time'),
        spark_sum('bytes_sent').alias('total_bytes')
    ) \
    .orderBy(desc('request_count'))

print("\n🔝 Топ-10 IP по количеству запросов:")
ip_stats.show(10)

## 5. Задание 2. Поиск аномалий с помощью Spark SQL

Используем метод 3-сигм (три стандартных отклонения) для выявления аномальных IP-адресов

In [ ]:
# Регистрация DataFrame как временное представление SQL
df_external.createOrReplaceTempView("access_logs")

print("✅ Временное представление 'access_logs' создано")

In [ ]:
# SQL-запрос для агрегации запросов по IP и времени
query_1 = """
SELECT 
    ip_address,
    DATE(timestamp) as date,
    HOUR(timestamp) as hour,
    COUNT(*) as request_count,
    AVG(response_time_ms) as avg_response_time,
    COUNT(DISTINCT endpoint) as unique_endpoints,
    SUM(CASE WHEN status_code >= 400 THEN 1 ELSE 0 END) as error_count
FROM access_logs
GROUP BY ip_address, DATE(timestamp), HOUR(timestamp)
ORDER BY request_count DESC
"""

ip_hourly_stats = spark.sql(query_1)

print("📊 Статистика по IP-адресам (почасовая):")
ip_hourly_stats.show(10)

In [ ]:
# SQL-запрос для выявления аномалий (метод 3-сигм)
query_2 = """
WITH ip_stats AS (
    SELECT 
        ip_address,
        COUNT(*) as total_requests,
        AVG(response_time_ms) as avg_response_time,
        COUNT(DISTINCT endpoint) as unique_endpoints,
        SUM(CASE WHEN status_code >= 400 THEN 1 ELSE 0 END) as error_count,
        MIN(timestamp) as first_seen,
        MAX(timestamp) as last_seen
    FROM access_logs
    GROUP BY ip_address
),
global_stats AS (
    SELECT 
        AVG(total_requests) as mean_requests,
        STDDEV(total_requests) as stddev_requests
    FROM ip_stats
),
anomaly_detection AS (
    SELECT 
        i.*,
        g.mean_requests,
        g.stddev_requests,
        (i.total_requests - g.mean_requests) / g.stddev_requests as z_score,
        CASE 
            WHEN (i.total_requests - g.mean_requests) / g.stddev_requests > 3 THEN 'CRITICAL'
            WHEN (i.total_requests - g.mean_requests) / g.stddev_requests > 2 THEN 'HIGH'
            WHEN (i.total_requests - g.mean_requests) / g.stddev_requests > 1 THEN 'MEDIUM'
            ELSE 'NORMAL'
        END as threat_level
    FROM ip_stats i
    CROSS JOIN global_stats g
)
SELECT *
FROM anomaly_detection
WHERE threat_level IN ('CRITICAL', 'HIGH')
ORDER BY z_score DESC
"""

anomalies = spark.sql(query_2)

print("🚨 Обнаруженные аномалии (IP с Z-score > 2):")
anomalies.show(20, truncate=False)

print(f"\n⚠️ Всего аномальных IP: {anomalies.count()}")

In [ ]:
# Детальный анализ аномальных IP
query_3 = """
SELECT 
    ip_address,
    endpoint,
    method,
    status_code,
    COUNT(*) as hit_count,
    AVG(response_time_ms) as avg_response_time
FROM access_logs
WHERE ip_address IN (
    SELECT ip_address 
    FROM (
        SELECT ip_address, COUNT(*) as cnt
        FROM access_logs
        GROUP BY ip_address
        HAVING COUNT(*) > (
            SELECT AVG(cnt) + 3 * STDDEV(cnt)
            FROM (
                SELECT COUNT(*) as cnt
                FROM access_logs
                GROUP BY ip_address
            )
        )
    )
)
GROUP BY ip_address, endpoint, method, status_code
ORDER BY ip_address, hit_count DESC
"""

anomaly_details = spark.sql(query_3)

print("🔍 Детальный анализ поведения аномальных IP:")
anomaly_details.show(30, truncate=False)

## 6. Задание 3. Визуализация результатов

In [ ]:
# Конвертация в Pandas для визуализации
# Временной ряд запросов
time_series_query = """
SELECT 
    timestamp,
    ip_address,
    CASE 
        WHEN ip_address IN (
            SELECT ip_address 
            FROM (
                SELECT ip_address, COUNT(*) as cnt
                FROM access_logs
                GROUP BY ip_address
                HAVING COUNT(*) > (
                    SELECT AVG(cnt) + 3 * STDDEV(cnt)
                    FROM (
                        SELECT COUNT(*) as cnt
                        FROM access_logs
                        GROUP BY ip_address
                    )
                )
            )
        ) THEN 1 ELSE 0
    END as is_anomaly
FROM access_logs
ORDER BY timestamp
"""

time_series_df = spark.sql(time_series_query).toPandas()
time_series_df['timestamp'] = pd.to_datetime(time_series_df['timestamp'])

print(f"✅ Подготовлено {len(time_series_df):,} записей для визуализации")

In [ ]:
# Визуализация 1: Временной ряд запросов с выделением аномалий
fig, ax = plt.subplots(figsize=(16, 6))

# Группировка по часам
hourly_requests = time_series_df.set_index('timestamp').resample('1H').size()
anomaly_requests = time_series_df[time_series_df['is_anomaly'] == 1] \
    .set_index('timestamp').resample('1H').size()

# Основная линия
ax.plot(hourly_requests.index, hourly_requests.values, 
        color='#2E86AB', linewidth=2, label='Общий трафик', alpha=0.8)

# Аномалии
ax.plot(anomaly_requests.index, anomaly_requests.values, 
        color='#E63946', linewidth=2.5, marker='o', markersize=5,
        label='Аномальный трафик (Z-score > 3σ)', linestyle='--')

# Области аномалий
ax.fill_between(anomaly_requests.index, 0, anomaly_requests.values, 
                alpha=0.3, color='#E63946')

# Средняя линия
mean_requests = hourly_requests.mean()
ax.axhline(y=mean_requests, color='#06A77D', linestyle=':', 
           linewidth=2, label=f'Среднее значение: {mean_requests:.0f}')

# Пороговая линия (mean + 3σ)
threshold = mean_requests + 3 * hourly_requests.std()
ax.axhline(y=threshold, color='#F77F00', linestyle='--', 
           linewidth=2, label=f'Порог аномалии (μ+3σ): {threshold:.0f}')

ax.set_xlabel('Время', fontsize=12, fontweight='bold')
ax.set_ylabel('Количество запросов в час', fontsize=12, fontweight='bold')
ax.set_title('Временной ряд запросов с выделением аномальной активности\n' +
             'Вариант 40: Кибербезопасность - Анализ логов доступа',
             fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3, linestyle=':')

plt.xticks(rotation=45, ha='right')
plt.tight_layout()

# ИСПРАВЛЕНО: сохранение в /tmp/
plt.savefig('/tmp/timeseries_anomalies.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ График сохранен: /tmp/timeseries_anomalies.png")

In [ ]:
# Визуализация 2: Тепловая карта активности по дням недели и часам
time_series_df['day_of_week'] = time_series_df['timestamp'].dt.day_name()
time_series_df['hour'] = time_series_df['timestamp'].dt.hour

# Создание сводной таблицы
heatmap_data = time_series_df.groupby(['day_of_week', 'hour']).size().unstack(fill_value=0)

# Правильный порядок дней недели
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data = heatmap_data.reindex([d for d in day_order if d in heatmap_data.index])

fig, ax = plt.subplots(figsize=(16, 6))
sns.heatmap(heatmap_data, cmap='YlOrRd', annot=True, fmt='d', 
            cbar_kws={'label': 'Количество запросов'},
            linewidths=0.5, linecolor='white', ax=ax)

ax.set_xlabel('Час дня', fontsize=12, fontweight='bold')
ax.set_ylabel('День недели', fontsize=12, fontweight='bold')
ax.set_title('Тепловая карта активности: День недели × Час',
             fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()
plt.savefig('/tmp/heatmap_activity.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ График сохранен: /tmp/heatmap_activity.png")

In [ ]:
# Визуализация 3. Топ аномальных IP-адресов
anomalies_pd = anomalies.toPandas()
top_anomalies = anomalies_pd.nlargest(10, 'total_requests')

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# График 1: Количество запросов
colors = ['#E63946' if level == 'CRITICAL' else '#F77F00' 
          for level in top_anomalies['threat_level']]

ax1.barh(range(len(top_anomalies)), top_anomalies['total_requests'], color=colors)
ax1.set_yticks(range(len(top_anomalies)))
ax1.set_yticklabels(top_anomalies['ip_address'])
ax1.set_xlabel('Количество запросов', fontsize=11, fontweight='bold')
ax1.set_title('Топ-10 аномальных IP по активности', fontsize=12, fontweight='bold')
ax1.grid(axis='x', alpha=0.3, linestyle=':')

# Добавление значений
for i, v in enumerate(top_anomalies['total_requests']):
    ax1.text(v + 50, i, f'{v:,}', va='center', fontsize=9, fontweight='bold')

# График 2: Z-Score
ax2.barh(range(len(top_anomalies)), top_anomalies['z_score'], color=colors)
ax2.set_yticks(range(len(top_anomalies)))
ax2.set_yticklabels(top_anomalies['ip_address'])
ax2.set_xlabel('Z-Score (σ)', fontsize=11, fontweight='bold')
ax2.set_title('Уровень аномальности (стандартные отклонения)', fontsize=12, fontweight='bold')
ax2.axvline(x=3, color='red', linestyle='--', linewidth=2, label='Порог (3σ)', alpha=0.7)
ax2.grid(axis='x', alpha=0.3, linestyle=':')
ax2.legend()

# Добавление значений
for i, v in enumerate(top_anomalies['z_score']):
    ax2.text(v + 0.2, i, f'{v:.1f}σ', va='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.savefig('/tmp/heatmap_activity.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ График сохранен: /tmp/heatmap_activity.png")

In [ ]:
# Визуализация 4: Scatter plot - Количество запросов vs Средняя задержка
ip_stats_pd = ip_stats.toPandas()

# Определение аномальных IP
mean_requests = ip_stats_pd['request_count'].mean()
std_requests = ip_stats_pd['request_count'].std()
threshold = mean_requests + 3 * std_requests

ip_stats_pd['is_anomaly'] = ip_stats_pd['request_count'] > threshold

fig, ax = plt.subplots(figsize=(14, 8))

# Нормальные IP
normal = ip_stats_pd[~ip_stats_pd['is_anomaly']]
ax.scatter(normal['request_count'], normal['avg_response_time'], 
           s=100, c='#2E86AB', alpha=0.6, label='Нормальный IP', edgecolors='white', linewidth=1.5)

# Аномальные IP
anomaly = ip_stats_pd[ip_stats_pd['is_anomaly']]
ax.scatter(anomaly['request_count'], anomaly['avg_response_time'], 
           s=300, c='#E63946', alpha=0.8, label='Аномальный IP (>3σ)', 
           edgecolors='darkred', linewidth=2, marker='D')

# Аннотации для аномалий
for idx, row in anomaly.iterrows():
    ax.annotate(row['ip_address'], 
                xy=(row['request_count'], row['avg_response_time']),
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.7),
                fontsize=8, fontweight='bold',
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0', color='black'))

# Пороговая линия
ax.axvline(x=threshold, color='#F77F00', linestyle='--', linewidth=2, 
           label=f'Порог аномалии: {threshold:.0f}', alpha=0.7)

ax.set_xlabel('Количество запросов от IP', fontsize=12, fontweight='bold')
ax.set_ylabel('Средняя задержка ответа (мс)', fontsize=12, fontweight='bold')
ax.set_title('Анализ корреляции: Активность IP vs Производительность\n' +
             'Scatter Plot с выделением аномальных паттернов',
             fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', fontsize=11)
ax.grid(True, alpha=0.3, linestyle=':')

plt.tight_layout()
plt.savefig('/tmp/heatmap_activity.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ График сохранен: /tmp/heatmap_activity.png")

## 7. Итоговая аналитика и выводы

In [ ]:
# Финальный отчет
print("="*80)
print("📊 ИТОГОВЫЙ ОТЧЕТ: Анализ безопасности логов доступа")
print("="*80)

# Общая статистика
total_records = df_clean.count()
external_records = df_external.count()
unique_ips = df_external.select('ip_address').distinct().count()
anomalous_ips = anomalies.count()

print(f"\n📈 Общая статистика:")
print(f"  • Всего записей: {total_records:,}")
print(f"  • Внешних IP: {external_records:,} ({external_records/total_records*100:.1f}%)")
print(f"  • Уникальных IP: {unique_ips:,}")
print(f"  • Аномальных IP: {anomalous_ips} ({anomalous_ips/unique_ips*100:.1f}%)")

# Топ аномальные IP
print(f"\n🚨 Топ-5 аномальных IP-адресов:")
top_5 = anomalies_pd.nlargest(5, 'z_score')[['ip_address', 'total_requests', 'z_score', 'threat_level']]
for idx, row in top_5.iterrows():
    print(f"  • {row['ip_address']: <18} | Запросов: {row['total_requests']:>6,} | "
          f"Z-score: {row['z_score']:>6.2f}σ | Угроза: {row['threat_level']}")

# Статистика по статус-кодам
status_stats = df_external.groupBy('status_code').count() \
    .orderBy(desc('count')).toPandas()

print(f"\n📡 Распределение HTTP статус-кодов:")
for _, row in status_stats.head(5).iterrows():
    print(f"  • {row['status_code']}: {row['count']:>8,} запросов")

# Рекомендации
print(f"\n💡 Рекомендации по безопасности:")
print(f"  1. Заблокировать {anomalous_ips} IP-адресов с аномальной активностью")
print(f"  2. Настроить rate limiting для предотвращения DDoS-атак")
print(f"  3. Внедрить CAPTCHA для подозрительных IP")
print(f"  4. Настроить алерты для IP с Z-score > 3σ")
print(f"  5. Провести детальный анализ endpoint'ов, подвергшихся атакам")

print("\n" + "="*80)
print("✅ Анализ завершен успешно!")
print("="*80)

bash# В терминале от пользователя hadoop:
sudo su - hadoop

# Дать права на запись пользователю devops
hdfs dfs -chmod -R 777 /user/hadoop/lab_01_variant40
# или создать директорию для devops
hdfs dfs -mkdir -p /user/devops/lab_01_variant40/output
hdfs dfs -chown -R devops:devops /user/devops/lab_01_variant40

In [ ]:
# Сохранение результатов в HDFS
output_path_anomalies = "hdfs://localhost:9000/user/hadoop/lab_01_variant40/output/anomalies"
output_path_stats = "hdfs://localhost:9000/user/hadoop/lab_01_variant40/output/ip_stats"

# Сохранение аномалий
anomalies.write.mode('overwrite') \
    .option("header", "true") \
    .csv(output_path_anomalies)

# Сохранение статистики
ip_stats.write.mode('overwrite') \
    .option("header", "true") \
    .csv(output_path_stats)

print("✅ Результаты сохранены в HDFS:")
print(f"  • {output_path_anomalies}")
print(f"  • {output_path_stats}")

# Сохранение результатов локально (не в HDFS)
output_path_anomalies = "/tmp/anomalies"
output_path_stats = "/tmp/ip_stats"

# Сохранение аномалий
anomalies.toPandas().to_csv('/tmp/anomalies.csv', index=False)

# Сохранение статистики
ip_stats.toPandas().to_csv('/tmp/ip_stats.csv', index=False)

print("✅ Результаты сохранены локально:")
print(f"  • /tmp/anomalies.csv")
print(f"  • /tmp/ip_stats.csv")

# Проверка
!ls -lh /tmp/anomalies.csv /tmp/ip_stats.csv
!head -5 /tmp/anomalies.csv

In [ ]:
# Остановка SparkSession
spark.stop()
print("✅ SparkSession остановлен")